<h1><center>University of Edinburgh Geomagnetism Tutorial</center></h1>

<h1><center>Spherical Harmonic Models 2</center></h1>

In [ ]:
# Import notebook dependencies

import sys
sys.path.append('..')
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import numpy as np
import pandas as pd 
from src import sha_lib as sha
import os

# 1. Another example of modelling using spherical harmonics

What happens if we tried to model the land/sea variation of the Earth.

First, there's a data file with some spatial data to plot.

In [ ]:
# Load the data
lsfile = '../external/land_5deg.csv'
lsdata = np.genfromtxt(lsfile, delimiter=',')

x = lsdata[:,0].reshape(36, 72)
y = lsdata[:,1].reshape(36,72)
z = lsdata[:,2].reshape(36,72)

# Plot them
plt.rcParams['figure.figsize'] = [15, 8]
plt.contourf(x,y,z)
plt.colorbar()
plt.show()

The data file consists of binary values (1=land, 0=water) on a 5-degree grid. We now take these data as the input to a global spherical harmonic analysis and calculate a spherical harmonic model with a user specified resolution.

### >> USER INPUT HERE: Set the maximum spherical harmonic degree of the analysis
What is a reasonable value? Try different values e.g. low degree (nmax = 5), medium degree (nmax = 13), high degree (nmax > 20)

In [ ]:
nmax = 5 # Max degree

In [ ]:
# These functions are used for the spherical harmonic analysis and synthesis steps
# Firstly compute a spherical harmonic model of degree and order nmax using the input data as plotted above
def sh_analysis(lsdata, nmax):
    npar = (nmax+1)*(nmax+1)
    ndat = len(lsdata)
    lhs  = np.zeros(npar*ndat).reshape(ndat,npar)

    rhs  = np.zeros(ndat)
    line = np.zeros(npar)
    ic   = -1
    for i in range(ndat):
        th  = 90 - lsdata[i][1]
        ph  = lsdata[i][0] 
        rhs[i] = lsdata[i][2]
        cmphi, smphi = sha.csmphi(nmax,ph)
        pnm = sha.pnm_calc(nmax, th)
        for n in range(nmax+1):
            igx = sha.gnmindex(n,0)
            ipx = sha.pnmindex(n,0)
            line[igx] = pnm[ipx]
            for m in range(1,n+1):
                igx = sha.gnmindex(n,m)
                ihx = sha.hnmindex(n,m)
                ipx = sha.pnmindex(n,m)
                line[igx] = pnm[ipx]*cmphi[m]
                line[ihx] = pnm[ipx]*smphi[m]
        lhs[i,:] = line

    shmod  = np.linalg.lstsq(lhs, rhs.reshape(len(lsdata),1), rcond=None)
    return(shmod)


# Now use the model to synthesise values on a 5 degree grid in latitude and longitude
def sh_synthesis(shcofs, nmax):
    newdata =np.zeros(72*36*3).reshape(2592,3)
    ic = 0
    for ilat in range(36):
        delta = 5*ilat+2.5
        lat = 90 - delta
        for iclt in range(72):
            corr  = 5*iclt+2.5
            long  = -180+corr
            colat = 90-lat
            cmphi, smphi = sha.csmphi(nmax,long)
            vals  = np.dot(sha.gh_phi(shcofs, nmax, cmphi, smphi), sha.pnm_calc(nmax, colat))
            newdata[ic,0]=long
            newdata[ic,1]=lat
            newdata[ic,2]=vals
            ic += 1
    return(newdata)

In [ ]:
# Obtain the spherical harmonic coefficients
shmod = sh_analysis(lsdata=lsdata, nmax=nmax)

# Read the model coefficients
shcofs = shmod[0]

# Synthesise the model coefficients on a 5 degree grid
newdata = sh_synthesis(shcofs=shcofs, nmax=nmax)
# Reshape for plotting purposes
x = newdata[:,0].reshape(36, 72)
y = newdata[:,1].reshape(36,72)
z = newdata[:,2].reshape(36,72)

# Plot the results
plt.rcParams['figure.figsize'] = [15, 8]
levels = [-1.5, -0.75, 0, 0.25, 0.5, 0.75, 2.]
plt.contourf(x,y,z)
plt.colorbar()
plt.show()

# 2. Spherical harmonic models with data gaps

What happens if the data set is incomplete? In this section, you can experiment by removing data within a great circle of a specified radius and position. The functions below are used to create the data gap, and the modelling uses functions in the above section again.

In [ ]:
def greatcircle(th1, ph1, th2, ph2):
    th1 = np.deg2rad(th1)
    th2 = np.deg2rad(th2)
    dph = np.deg2rad(dlong(ph1,ph2))

  # Apply the cosine rule of spherical trigonometry
    dth = np.arccos(np.cos(th1)*np.cos(th2) + \
              np.sin(th1)*np.sin(th2)*np.cos(dph))
    return(dth)

def dlong (ph1, ph2):
    ph1 = np.sign(ph1)*abs(ph1)%360   # These lines return a number in the 
    ph2 = np.sign(ph2)*abs(ph2)%360   # range -360 to 360
    if(ph1 < 0): ph1 = ph1 + 360      # Put the results in the range 0-360
    if(ph2 < 0): ph2 = ph2 + 360
    dph = max(ph1,ph2) - min(ph1,ph2) # So the answer is positive and in the
                                      # range 0-360
    if(dph > 180): dph = 360-dph      # So the 'short route' is returned
    return(dph)

def gh_phi(gh, nmax, cp, sp):
    rx = np.zeros(nmax*(nmax+3)//2+1)
    igx=-1
    igh=-1
    for i in range(nmax+1):
        igx += 1
        igh += 1
        rx[igx]= gh[igh]
        for j in range(1,i+1):
            igh += 2
            igx += 1
            rx[igx] = (gh[igh-1]*cp[j] + gh[igh]*sp[j])
    return(rx)

### >> USER INPUT HERE: Set the location and size of the data gap

Colatitude: degrees

Longitude: degrees

Radius: km

In [ ]:
# Remove a section of data centered on colat0, long0 and radius here
colat0 = 100
long0  = -55
radius  = 5000

In [ ]:
lsdata_gap = lsdata.copy()
for row in range(len(lsdata_gap)):
    colat = 90 - lsdata_gap[row,1]
    long  = lsdata_gap[row,0]
    if greatcircle(colat, long, colat0, long0) < radius/6371.2:
        lsdata_gap[row,2] = np.nan

print('Blanked out: ', np.count_nonzero(np.isnan(lsdata_gap)))

x_gap = lsdata_gap[:,0].reshape(36, 72)
y_gap = lsdata_gap[:,1].reshape(36,72)
z_gap = lsdata_gap[:,2].reshape(36,72)

# Plot the map with omitted data
plt.contourf(x_gap, y_gap, z_gap)

### >> USER INPUT HERE: Set the maximum spherical harmonic degree of the analysis

In [ ]:
nmax = 13 #Max degree

In [ ]:
# Select the non-nan data
lsdata_gap = lsdata_gap[~np.isnan(lsdata_gap[:,2])]

# Obtain the spherical harmonic coefficients for the incomplete data set
shmod = sh_analysis(lsdata=lsdata_gap, nmax=nmax)

# Read the model coefficients
shcofs = shmod[0]

# Synthesise the model coefficients on a 5 degree grid
newdata = sh_synthesis(shcofs=shcofs, nmax=nmax)
# Reshape for plotting purposes
x_new = newdata[:,0].reshape(36, 72)
y_new = newdata[:,1].reshape(36,72)
z_new = newdata[:,2].reshape(36,72)

Print the maximum and minimum of the synthesised data. How do they compare to the original data, which was composed of only ones and zeroes? How do the max/min change as you vary the data gap size and the analysis resolution? Try this with a large gap, e.g. 5000 km.

In [ ]:
print(np.min(z_new))
print(np.max(z_new))

In [ ]:
# Plot the results with colour scale according to the data
plt.rcParams['figure.figsize'] = [15, 8]
plt.contourf(x_new, y_new, z_new)
plt.colorbar()
plt.show()

Now see what happens when we restrict the colour scale to values between 0 and 1 regardless of the data values (so that both ends of the colour scale are saturated).

In [ ]:
# Plot the results
plt.rcParams['figure.figsize'] = [15, 8]
levels = [0, 0.25, 0.5, 0.75, 1.]
plt.contourf(x_new, y_new, z_new, levels, extend='both')
plt.colorbar()
plt.show()

### Final questions
1. What do the reconstructions of the land and sea tell us about the use of spherical harmonics? Why do we still use them?
2. How much can we trust the smaller scale features of the IGRF or any other main field model? 
3. How do you think bad data affect the magnitude of the field values we recover at the core-mantle boundary?

### Acknowledgements

The land/sea data file was provided by John Stevenson (BGS).
